## part 1

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 

soup = BeautifulSoup(source,'lxml')

table = soup.find('table', class_= 'wikitable sortable')
row = table.find_all('tr')

list_1 = []

for tr in row:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    list_1.append(row)
    

df = pd.DataFrame(list_1, columns = ["Postalcode", "Borough", "Neighbourhood"])
df = df[df.Borough != "Not assigned"]
df[df.Neighbourhood == "Not assigned"].Neighbourhood = df.Borough
df = df.groupby(['Postalcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

df
df.shape

(103, 3)

## part 2

In [5]:
df_2 = pd.read_csv('http://cocl.us/Geospatial_data')
df_2.columns = ["Postalcode", "Latitude", "Longitude"]
df3 = pd.merge(left = df, right = df_2, left_on = "Postalcode", right_on = "Postalcode")

df3

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## part 3

In [11]:
from geopy.geocoders import Nominatim
!pip install folium
import folium

address = "Toronto, Ontario"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_t = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_t) 
map_t

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
